In [ ]:
pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
!pip install _pywrap_tensorflow_internal

In [ ]:
import tensorflow

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd

dirs = "C:/Users/H/PetImages"
categories = ['Dog','Cat']

for category in categories:
    path = os.path.join(dirs,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        plt.imshow(img_array,cmap = 'gray')
        plt.show()
        break
    break

In [ ]:
print(img_array.shape)

In [ ]:
training_data = []
img_size=50
def create_training_data():
    for category in categories:
        path = os.path.join(dirs,category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array,(50,50))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
            
create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
import random
random.shuffle(training_data)

In [ ]:
import numpy as np
x = []
y = []

for features, label in training_data:
    x.append(features)
    y.append(label)
x = np.array(x).reshape(-1,50,50,1)
y = np.array(y).reshape(-1)

In [ ]:
import pickle
pickle_out = open("x.pickle","wb")
pickle.dump(x,pickle_out)
pickle_out.close

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle
import time





x = pickle.load(open("x.pickle","rb"))
#y = pickle.load(open("y.pickle","rb"))

x = x/255.0

dense_layers = [0,1,2]
layer_sizes = [32,64,128]
conv_layers = [1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            name = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            tensorboard = TensorBoard(log_dir = 'C:\\Users\\H\\logs\\{}'.format(name),profile_batch = 100000000)
            model = Sequential()

            model.add(Conv2D(layer_size,(3,3),input_shape = x.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten())
            
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))

            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
            model.fit(x,y,batch_size=32,epochs = 10,validation_split = .3, callbacks = [tensorboard])
            
model.save('dc.model')